In [1]:
# ==================== Tip: Would restart colab session.Waiting then restarting ====================
# Loss view
!pip install tbparse
# Upgrade pip and install core Python dependencies
!pip install --upgrade pip setuptools wheel
!pip install pyyaml transformers datasets einops Werkzeug==2.0.3 flask==2.0.3 revtok vocab
!pip install ai2thor==4.3.0 pyvirtualdisplay
!pip install tensorboardX
# Update and install the display dependencies
!apt-get update -y
!apt-get install -y git wget unzip ffmpeg libgl1-mesa-glx libglib2.0-0 xvfb
# huggingface load
!pip install datasets huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 72.9 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: flask
    Found existing installation: Flask 3.1.2
    Uninstalling Flask-3.1.2:
      Successfully uninstalled Flask-3.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [flask]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 174.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [ai2thor]
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repo

In [1]:
# ==================== Basics: GPU, CUDA, cuDNN, OS ====================
import os, json, platform, torch, subprocess, sys, shutil

def sh(cmd):
    return subprocess.check_output(cmd, shell=True, text=True, stderr=subprocess.STDOUT).strip()

gpu = sh("nvidia-smi --query-gpu=name,memory.total --format=csv,noheader") if shutil.which("nvidia-smi") else "N/A"
cuda = torch.version.cuda
cudnn = torch.backends.cudnn.version()


os_release = sh("cat /etc/os-release || true")
os_line = next((l for l in os_release.splitlines() if l.startswith("PRETTY_NAME=")), "PRETTY_NAME=Unknown")
os_pretty = os_line.split("=",1)[1].strip().strip('"')

# --- Python packages ---
pyver = sys.version.split()[0]
torchver = torch.__version__
try:
    import transformers
    tfver = transformers.__version__
except Exception:
    tfver = "not installed"

try:
    import huggingface_hub
    hfver = huggingface_hub.__version__
except Exception:
    hfver = "not installed"

try:
    import ai2thor
    thorver = getattr(ai2thor, "__version__", "unknown")
except Exception:
    thorver = "not installed"

hf_repo = "byeonghwikim/abp_dataset"
dataset_sha = "unknown"
try:
    from huggingface_hub import HfApi
    api = HfApi()
    info = api.dataset_info(hf_repo)
    dataset_sha = getattr(info, "sha", "unknown")
except Exception as e:
    dataset_sha = f"unknown ({e})"

try:
    from huggingface_hub import hf_hub_url
    url = hf_hub_url(hf_repo, "README.md", repo_type="dataset")

    if "/resolve/" in url:
        dataset_sha_from_url = url.split("/resolve/")[1].split("/")[0]
    else:
        dataset_sha_from_url = "unknown"
except Exception:
    dataset_sha_from_url = "unknown"

env = {
    "GPU": gpu,
    "CUDA": cuda,
    "cuDNN": cudnn,
    "OS": os_pretty,
    "Python": pyver,
    "PyTorch": torchver,
    "Transformers": tfver,
    "huggingface_hub": hfver,
    "AI2-THOR": thorver,
    "HF_Dataset": hf_repo,
    "HF_Dataset_SHA_HEAD": dataset_sha,
    "HF_Dataset_SHA_from_url": dataset_sha_from_url,
}

print(json.dumps(env, indent=2))


os.makedirs("/content/env_dump", exist_ok=True)
with open("/content/env_dump/run_config_env.json", "w") as f:
    json.dump(env, f, indent=2)

print("\nSaved to /content/env_dump/run_config_env.json")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{
  "GPU": "Tesla T4, 15360 MiB",
  "CUDA": "12.6",
  "cuDNN": 91002,
  "OS": "Ubuntu 22.04.4 LTS",
  "Python": "3.12.11",
  "PyTorch": "2.8.0+cu126",
  "Transformers": "4.56.1",
  "huggingface_hub": "0.34.4",
  "AI2-THOR": "4.3.0",
  "HF_Dataset": "byeonghwikim/abp_dataset",
  "HF_Dataset_SHA_HEAD": "52d062450312ad6fd8970ef6a469888f6766a661",
  "HF_Dataset_SHA_from_url": "main"
}

Saved to /content/env_dump/run_config_env.json


In [2]:
# ==================== clone & set env variances  ====================

# Clone repository
!git clone https://github.com/ruohuaqian/mcr-agent.git /content/mcr-agent
# Set environment variables
%env MCR_ROOT=/content/mcr-agent
# %env PYTHONPATH = /content/alfred:/content/mcr-agent:${PYTHONPATH}
# --- set path of project ---
import sys
import os
import shutil
#for rollout

%cd /content/mcr-agent/models
!wget https://alfred-colorswap.s3.us-east-2.amazonaws.com/weight_maskrcnn.pt

# Make ALFRED and MCR paths take precedence
for p in [
    "/content/mcr-agent",                               # MCR root
]:
    if p not in sys.path:
        sys.path.insert(0, p)

Cloning into '/content/mcr-agent'...
remote: Enumerating objects: 3157, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 3157 (delta 119), reused 95 (delta 76), pack-reused 3007 (from 1)
Receiving objects: 100% (3157/3157), 1.87 MiB | 26.55 MiB/s, done.
Resolving deltas: 100% (2356/2356), done.
env: MCR_ROOT=/content/mcr-agent
/content/mcr-agent/models
--2025-09-19 07:35:54--  https://alfred-colorswap.s3.us-east-2.amazonaws.com/weight_maskrcnn.pt
Resolving alfred-colorswap.s3.us-east-2.amazonaws.com (alfred-colorswap.s3.us-east-2.amazonaws.com)... 3.5.133.160, 3.5.132.21, 3.5.131.133, ...
Connecting to alfred-colorswap.s3.us-east-2.amazonaws.com (alfred-colorswap.s3.us-east-2.amazonaws.com)|3.5.133.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 178697150 (170M) [application/x-www-form-urlencoded]
Saving to: ‘weight_maskrcnn.pt’

weight_maskrcnn.pt  100%[===================>] 170.42M  50.5

In [3]:
# ==================== ai2thor test ====================
from pyvirtualdisplay import Display
# Standardize on 24-bit color depth to prevent Unity/GLX color depth compatibility pitfalls
display = Display(visible=0, size=(1024, 768), color_depth=24)
display.start()

import os
os.environ["SDL_AUDIODRIVER"] = "dummy"

# ai2thor test
from ai2thor.controller import Controller

controller = Controller(
    scene="FloorPlan1",
    width=300, height=300,
    renderDepthImage=False,
    renderClassImage=False,
    renderObjectImage=False,
)
event = controller.step(action="Initialize", gridSize=0.25)
print("Initialized:", event.metadata["lastActionSuccess"])
controller.stop()


thor-Linux64-a634e28ea55b149ce5f746f19f45dd6f43e5878b.zip: [ 100%  91.9 MiB/s]  of 529.MB


Initialized: True


In [4]:
# ==================== Set up a compatible environment&files. ====================

# Uninstall old versions to prevent conflicts.
%pip -q uninstall -y torch torchvision torchaudio

# Install a recent, Py3.12-friendly Torch stack with a pinned NumPy version.
# Use the CUDA 12.1 index for Colab's GPU runtime.

%pip -q install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121

# Verify versions.
import torch
import numpy as np

print(f"Torch: {torch.__version__} | NumPy: {np.__version__}")
x = torch.randn(1)
print(f"Torch works: {x.shape}")

import os
import re

def patch_directory(directory, pattern, replacement):
    """
    Recursively finds and replaces text in all .py files within a directory.
    """
    for root, _, files in os.walk(directory):
        for fname in files:
            if fname.endswith(".py"):
                fpath = os.path.join(root, fname)
                with open(fpath, "r") as f:
                    text = f.read()

                new_text = re.sub(pattern, replacement, text)

                if new_text != text:
                    with open(fpath, "w") as f:
                        f.write(new_text)
                    print(f"Patched {fpath}")

# Fix 'torch._six.int_classes' to 'int_classes = (int,)'.
int_classes_replacements = {
    "from torch._six import int_classes": "int_classes = (int,)"
}

# Fix 'dict.has_key' to 'in' keyword using a regular expression.
patch_directory(
    "/content/alfred",
    r"(\w+)\.has_key\((.*?)\)",
    r"\2 in \1"
)
patch_directory(
    "/content/mcr-agent",
    r"(\w+)\.has_key\((.*?)\)",
    r"\2 in \1"
)

Torch: 2.8.0+cu126 | NumPy: 2.0.2
Torch works: torch.Size([1])


In [5]:
# ==================== Mount Google Drive & Download files ====================
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

%cd /content/mcr-agent

import os
import sys

# --- Check and use data in Google Drive ---
# DRIVE_DATASET_PATH = "/content/drive/MyDrive/json_feat_2.1.0"
# LOCAL_DATASET_PATH = "/content/mcr-agent/data/json_feat_2.1.0"
DRIVE_SPLITS_PATH = "/content/drive/MyDrive/mcr-agent/splits"
LOCAL_SPLITS_PATH = "/content/mcr-agent/data/splits"
DRIVE_OUTPUT_PATH = "/content/drive/MyDrive/mcr-agent/exp"
LOCAL_OUTPUT_PATH = "/content/mcr-agent/exp"

# Create necessary directories
!mkdir -p "/content/drive/MyDrive/mcr-agent"

# Check if dataset exists in Google Drive
# if os.path.exists(DRIVE_DATASET_PATH) and os.listdir(DRIVE_DATASET_PATH):
#     print("Using existing dataset in Google Drive")
# else:
#     print("Dataset not found in Google Drive, downloading directly to Google Drive...")
#     # Download directly to Google Drive
#     %cd "/content/drive/MyDrive"
#     !git clone https://huggingface.co/datasets/byeonghwikim/abp_dataset json_feat_2.1.0
#     print("Dataset downloaded and extracted directly to Google Drive")

# Create symlink from Google Drive to local
# !ln -sf "{DRIVE_DATASET_PATH}" "{LOCAL_DATASET_PATH}"

# Handle splits data
!mkdir -p "{DRIVE_SPLITS_PATH}"    # Create splits directory in Google Drive
if os.path.exists(DRIVE_SPLITS_PATH) and os.listdir(DRIVE_SPLITS_PATH):
    print("Using existing splits data in Google Drive")
else:
    print("Downloading splits data...")
    # Download splits directly to Google Drive
    %cd "{DRIVE_SPLITS_PATH}"
    !wget https://raw.githubusercontent.com/askforalfred/alfred/master/data/splits/oct21.json
    print("Splits data downloaded to Google Drive and copied to local")

# Validate data
print("Data directory structure:")
!ls -l /content/mcr-agent/data

# print("\nGoogle Drive dataset size:")
# !du -sh "{DRIVE_DATASET_PATH}" 2>/dev/null || echo "Checking dataset size..."

# Create output directory structure
!mkdir -p "{DRIVE_OUTPUT_PATH}"
!mkdir -p "{LOCAL_OUTPUT_PATH}"

print("\nSetup completed! Dataset is stored in Google Drive:")
# print(f"Dataset: {DRIVE_DATASET_PATH}")
print(f"Splits: {DRIVE_SPLITS_PATH}")
print(f"Output will be saved to: {DRIVE_OUTPUT_PATH}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/mcr-agent
Using existing splits data in Google Drive
Data directory structure:
total 16
-rw-r--r-- 1 root root 1117 Sep 19 07:35 download_data.sh
-rw-r--r-- 1 root root 9056 Sep 19 07:35 preprocess.py

Setup completed! Dataset is stored in Google Drive:
Splits: /content/drive/MyDrive/mcr-agent/splits
Output will be saved to: /content/drive/MyDrive/mcr-agent/exp


In [6]:
# ==================== Split the split json to pretrain/finetune ====================

import json
from collections import defaultdict
import random

def seperate_data(original_data, postpartum_objects):
   # Define relevant actions and destinations
    relevant_actions = [
        'pick_clean_then_place_in_recep',
        'pick_two_obj_and_place',
        'pick_and_place_simple'
    ]
    relevant_destinations = [
        'Toilet', 'BathtubBasin', 'ArmChair',
        'Bed', 'SideTable', 'SinkBasin'
    ]
    list_postpartum = []
    list_rest = []
    trajectory_instances = defaultdict(list)
    object_trajectories = {obj: set() for obj in postpartum_objects}  # Track unique trajectories per object
    # Group tasks by trajectory (task name without trial path)
    for item in original_data:
        task_str = item.get('task', '')
        parts = task_str.split('/')

        if len(parts) > 0:
            try:
                parts_detail = parts[0].split('-')
                if len(parts_detail) >= 4:
                    action = parts_detail[0]
                    object_info = parts_detail[-4]
                    destination = parts_detail[-2]
                    scene_number = int(parts_detail[-1])

                    # Check if task is relevant for postpartum and scene number
                    if (object_info in postpartum_objects and
                        action in relevant_actions and
                        destination in relevant_destinations and
                        (scene_number in range(301, 331) or scene_number in range(401, 431))):
                        trajectory_name = task_str  # Task name without trial path
                        trajectory_instances[trajectory_name].append(item)
                        continue
            except Exception:
                pass
        list_rest.append(item)

    # Select up to max_per_object trajectories per object, including all instances
    for trajectory_name, instances in trajectory_instances.items():
        try:
            parts_detail = trajectory_name.split('-')
            object_info = parts_detail[-4]
            if object_info in postpartum_objects:
                # Add all instances of this trajectory to list_postpartum
                list_postpartum.extend(instances)
                object_trajectories[object_info].add(trajectory_name)
                # Remove all instances from list_rest
                for instance in instances:
                    if instance in list_rest:
                        list_rest.remove(instance)
        except Exception:
            continue


    # Calculate object counts for output (number of unique trajectories)
    object_counts = {obj: len(trajectories) for obj, trajectories in object_trajectories.items()}
    # Print count of unique trajectories per object type
    print("Postpartum object trajectory counts:")
    for obj, count in object_counts.items():
            print(f"  {obj}: {count}")
    return list_postpartum, list_rest
def create_k_shot_split(full_postpartum_train_data, k_shot):
    """
    shots from postpartum train data
    """
    # group by object
    object_to_trajectories = defaultdict(set)
    for item in full_postpartum_train_data:
        try:
            trajectory_name = item.get('task', '').split('/')[0]
            object_type = trajectory_name.split('-')[-4]
            object_to_trajectories[object_type].add(trajectory_name)
        except IndexError:
            continue

    # select K trajectories for each object
    selected_trajectories = set()
    for obj, trajectories in sorted(object_to_trajectories.items()):
        trajectories_list = list(trajectories)
        random.shuffle(trajectories_list)

        trajectories_to_add = trajectories_list[:k_shot]
        selected_trajectories.update(trajectories_to_add)
        print(f"  - '{obj}': find {len(trajectories_list)} trajectories，choose {len(trajectories_to_add)}.")

    # collect annotation
    k_shot_train_data = []
    for item in full_postpartum_train_data:
        trajectory_name = item.get('task', '').split('/')[0]
        if trajectory_name in selected_trajectories:
            k_shot_train_data.append(item)

    print(f"  - New {k_shot}-shot contains {len(k_shot_train_data)} annotations.")
    return k_shot_train_data


def filter_postpartum_tasks(dir, input_file, output_file_postpartum, output_file_rest, postpartum_objects):

    # Load the input JSON file
    try:
        with open(input_file, 'r') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Error: The file '{input_file}' was not found.")
        return [], []
    except json.JSONDecodeError:
        print(f"Error: The file '{input_file}' is not a valid JSON file.")
        return [], []

    list_postpartum_train, list_rest_train = seperate_data(data.get('train', []), postpartum_objects)
    list_postpartum_valid_seen, list_rest_valid_seen = seperate_data(data.get('valid_seen',[]), postpartum_objects)
    list_postpartum_valid_unseen, list_rest_valid_unseen = seperate_data(data.get('valid_unseen',[]), postpartum_objects)
    # Save to output JSON files
    try:
        with open(output_file_postpartum, 'w') as f:
          data['train'] = list_postpartum_train
          data['valid_seen'] = list_postpartum_valid_seen
          data['valid_unseen'] = list_postpartum_valid_unseen
          json.dump(data, f)
        with open(output_file_rest, 'w') as f:
          data['train'] = list_rest_train
          data['valid_seen'] = list_postpartum_valid_seen
          data['valid_unseen'] = list_postpartum_valid_unseen
          json.dump(data, f)
        print(f"Successfully saved {len(list_postpartum_train)} postpartum annotations to '{output_file_postpartum}'")
        print(f"Successfully saved {len(list_rest_train)} remaining annotations to '{output_file_rest}'")

        for k in [1, 5, 10]:
            k_shot_train_data = create_k_shot_split(list_postpartum_train, k)
            k_shot_data = {
                'train': k_shot_train_data,
                'valid_seen': list_postpartum_valid_seen,
                'valid_unseen': list_postpartum_valid_unseen
            }

            # save K-shot for finetune
            k_shot_output_path = os.path.join(dir, f'{k}-shot.json')

            with open(k_shot_output_path, 'w') as f:
                json.dump(k_shot_data, f, indent=2)
            print(f"successfully save {k}-shot split to '{k_shot_output_path}'")
        return None
    except Exception as e:
        print(f"Error: Could not save to files. Reason: {e}")

drive_splits_dir = "/content/drive/MyDrive/mcr-agent/splits"
input_file = drive_splits_dir + '/oct21.json'
fine_tune_split_file = drive_splits_dir + '/postpartum_train.json'
pretrain_split_file = drive_splits_dir + '/rest_train.json'
postpartum_objects = ["Cloth", "HandTowel", "Pillow", "SoapBar", "SoapBottle", "TissueBox"]

filter_postpartum_tasks(
      drive_splits_dir, input_file, fine_tune_split_file, pretrain_split_file, postpartum_objects
)

Postpartum object trajectory counts:
  Cloth: 93
  HandTowel: 62
  Pillow: 5
  SoapBar: 81
  SoapBottle: 44
  TissueBox: 9
Postpartum object trajectory counts:
  Cloth: 3
  HandTowel: 3
  Pillow: 0
  SoapBar: 7
  SoapBottle: 0
  TissueBox: 2
Postpartum object trajectory counts:
  Cloth: 0
  HandTowel: 0
  Pillow: 0
  SoapBar: 0
  SoapBottle: 3
  TissueBox: 0
Successfully saved 923 postpartum annotations to '/content/drive/MyDrive/mcr-agent/splits/postpartum_train.json'
Successfully saved 20100 remaining annotations to '/content/drive/MyDrive/mcr-agent/splits/rest_train.json'
  - 'Cloth': find 36 trajectories，choose 1.
  - 'HandTowel': find 23 trajectories，choose 1.
  - 'Pillow': find 2 trajectories，choose 1.
  - 'SoapBar': find 31 trajectories，choose 1.
  - 'SoapBottle': find 15 trajectories，choose 1.
  - 'TissueBox': find 4 trajectories，choose 1.
  - New 1-shot contains 56 annotations.
successfully save 1-shot split to '/content/drive/MyDrive/mcr-agent/splits/1-shot.json'
  - 'Cloth':

In [ ]:
# ==================== Train models separetely ====================
# prepare the temp dir
!mkdir -p /content/tmp /tmp /var /tmp /usr/tmp
!chmod 1777 /content/tmp /tmp /var/tmp /usr/tmp

%cd /content/mcr-agent
# ensure shell include true end
!sed -i 's/\r$//' /content/mcr-agent/PCC/train_pcc_stream.sh 2>/dev/null || echo "sed command failed, but continuing..."

# set temp dir env path
import os
os.environ['TMPDIR'] = '/content/tmp'
os.environ['TMP'] = '/content/tmp'
os.environ['TEMP'] = '/content/tmp'
###############################################
os.environ["HF_TOKEN"]  =  "" ####### HF_TOKEN NEEDED
###############################################

# delete the current local output dir(if existed)
!rm -rf "{LOCAL_OUTPUT_PATH}" 2>/dev/null || true
!sed -i 's/\r$//' /content/mcr-agent/OEM/train_oem.sh

# no panoramic
!bash /content/mcr-agent/Interactions/train_ip_stream.sh \
  --splits "/content/drive/MyDrive/mcr-agent/splits/rest_train.json" \
  --batch 16 \
  --epoch 5 \
  --lr 1e-3 \
  --dataset_fraction 1000 \
  --vis_dropout 0.3 \
  --hstate_dropout 0.3 \
  --gpu \
  --use_streaming \
  --dout /content/drive/MyDrive/mcr-agent/exp/Interactions

# no panoramic
!bash /content/mcr-agent/PCC/train_pcc_stream.sh \
  --splits "/content/drive/MyDrive/mcr-agent/splits/rest_train.json" \
  --batch 16 \
  --epoch 25 \
  --lr 1e-3 \
  --dataset_fraction 1000 \
  --vis_dropout 0.3 \
  --hstate_dropout 0.3 \
  --gpu \
  --dout /content/drive/MyDrive/mcr-agent/exp/PCC \

!bash /content/mcr-agent/MasterPolicy/train_mp_stream.sh \
  --splits "/content/drive/MyDrive/mcr-agent/splits/rest_train.json" \
  --batch 16 \
  --epoch 25 \
  --lr 1e-3 \
  --dataset_fraction 1000 \
  --vis_dropout 0.3 \
  --hstate_dropout 0.3 \
  --gpu \
  --panoramic \
  --orientation \
  --panoramic_concat \
  --dout /content/drive/MyDrive/mcr-agent/exp/MasterPolicy

!bash /content/mcr-agent/OEM/train_oem_stream.sh \
  --splits "/content/drive/MyDrive/mcr-agent/splits/rest_train.json" \
  --batch 16 \
  --epoch 25 \
  --lr 1e-3 \
  --dataset_fraction 1000 \
  --vis_dropout 0.3 \
  --hstate_dropout 0.3 \
  --panoramic \
  --orientation \
  --panoramic_concat \
  --gpu \
  --dout /content/drive/MyDrive/mcr-agent/exp/OEM



/content/mcr-agent
[INFO] Starting Training Run
------------------------------------------------------------------------
[INFO] Mode: Streaming
[INFO] Model: OEM.models.model.seq2seq_im_mask
[INFO] Output Dir: /content/drive/MyDrive/mcr-agent/exp/OEM
[INFO] Batch Size: 16
[INFO] Epochs: 25
------------------------------------------------------------------------
[CMD] python models/train/train_seq2seq_stream.py --seed 123 --splits /content/drive/MyDrive/mcr-agent/splits/rest_train.json --pp_folder pp --huggingface_id byeonghwikim/abp_dataset --model OEM.models.model.seq2seq_im_mask --dout /content/drive/MyDrive/mcr-agent/exp/OEM --batch 16 --epoch 25 --lr 1e-3 --decay_epoch 10 --dhid 512 --dframe 147 --demb 100 --pframe 300 --mask_loss_wt 1.0 --action_loss_wt 1.0 --subgoal_aux_loss_wt 0.2 --pm_aux_loss_wt 0.2 --lang_dropout 0.0 --input_dropout 0.0 --vis_dropout 0.3 --hstate_dropout 0.3 --attn_dropout 0.0 --actor_dropout 0.0 --gpu --use_streaming --panoramic --orientation --panoramic_con

In [7]:

from tbparse import SummaryReader
import pandas as pd

log_dir = '/content/drive/MyDrive/mcr-agent/exp/OEM'

def analyze_tensorboard_logs(log_dir: str) -> dict:
    try:
        reader = SummaryReader(log_dir)
        df = reader.scalars

        if df.empty:
            print(f"'{log_dir}' empty")
            return {}

        print(f"--- success load log from '{log_dir}' ---")

        train_loss_df = df.loc[df['tag'] == 'train/loss'].copy()
        valid_seen_loss_df = df.loc[df['tag'] == 'valid/seen_loss'].copy()
        valid_unseen_loss_df = df.loc[df['tag'] == 'valid/unseen_loss'].copy()

        loss_data = {
            'train': train_loss_df,
            'seen': valid_seen_loss_df,
            'unseen': valid_unseen_loss_df
        }


        return loss_data

    except Exception as e:
        print(f"error {e} when solve '{log_dir}' ")
        return {}

oem_log_dir = '/content/drive/MyDrive/mcr-agent/exp/OEM'
oem_loss_data = analyze_tensorboard_logs(oem_log_dir)
print(oem_loss_data['seen'].tail())
print(oem_loss_data['unseen'].tail())

--- success load log from '/content/drive/MyDrive/mcr-agent/exp/OEM' ---
        step              tag     value
30209  15792  valid/seen_loss  0.275856
30210  18424  valid/seen_loss  0.258759
30211  21056  valid/seen_loss  0.260709
30212  23688  valid/seen_loss  0.264969
30213  26320  valid/seen_loss  0.273013
        step                tag     value
30359  15792  valid/unseen_loss  0.152737
30360  18424  valid/unseen_loss  0.288188
30361  21056  valid/unseen_loss  0.470037
30362  23688  valid/unseen_loss  0.433503
30363  26320  valid/unseen_loss  0.508002


In [ ]:
# %cd /content/mcr-agent
# !git fetch
# !git reset --hard origin/main

/content/mcr-agent
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 10 (delta 8), reused 10 (delta 8), pack-reused 0 (from 0)
Unpacking objects: 100% (10/10), 1.08 KiB | 370.00 KiB/s, done.
From https://github.com/ruohuaqian/mcr-agent
   57fdeb4..b054d8f  main       -> origin/main
HEAD is now at b054d8f action_high_order None


In [ ]:
# ==================== Run the rollout script - 0 shot ====================
!bash /content/mcr-agent/valid_seen.sh
!bash /content/mcr-agent/valid_unseen.sh

{'tests_seen': 1533,
 'tests_unseen': 1529,
 'train': 20100,
 'valid_seen': 49,
 'valid_unseen': 10}
Loading Nav:  /content/drive/MyDrive/mcr-agent/exp/MasterPolicy/latest.pth
Loading Pickup:  /content/drive/MyDrive/mcr-agent/exp/Interactions/PickupObject/latest.pth
Loading put:  /content/drive/MyDrive/mcr-agent/exp/Interactions/PutObject/latest.pth
Loading heat:  /content/drive/MyDrive/mcr-agent/exp/Interactions/HeatObject/latest.pth
Loading cool:  /content/drive/MyDrive/mcr-agent/exp/Interactions/CoolObject/latest.pth
Loading clean:  /content/drive/MyDrive/mcr-agent/exp/Interactions/CleanObject/latest.pth
Loading toggle:  /content/drive/MyDrive/mcr-agent/exp/Interactions/ToggleObject/latest.pth
Loading slice:  /content/drive/MyDrive/mcr-agent/exp/Interactions/SliceObject/latest.pth
Loading object model:  /content/drive/MyDrive/mcr-agent/exp/OEM/latest.pth
Loading subgoal prediction model:  /content/drive/MyDrive/mcr-agent/exp/PCC/latest.pth
/usr/local/lib/python3.12/dist-packages/tor

In [ ]:
# ==================== 1-shot fine-tune ====================

!bash /content/mcr-agent/OEM/finetune.sh \
  --ckpt /content/drive/MyDrive/mcr-agent/exp/OEM/latest.pth \
  --out  /content/drive/MyDrive/mcr-agent/exp/OEM/ft_001 \
  --model OEM.models.model.seq2seq_im_mask_obj \
  --splits /content/drive/MyDrive/mcr-agent/splits/1-shot.json \
  --batch 1 --epochs 8 --lr 1e-4 \
  --freeze "encoder,visual_backbone,embeddings" \
  --extra "--dec_teacher_forcing"


[finetune] Launching:
python -u models/train/train_seq2seq_stream.py   --seed 123   --huggingface_id byeonghwikim/abp_dataset   --splits /content/drive/MyDrive/mcr-agent/splits/1-shot.json   --pp_folder pp   --model OEM.models.model.seq2seq_im_mask_obj   --gpu   --dout /content/drive/MyDrive/mcr-agent/exp/OEM/ft_001   --finetune_from /content/drive/MyDrive/mcr-agent/exp/OEM/latest.pth   --use_streaming   --batch 1   --epoch 8   --lr 1e-4   --freeze "encoder,visual_backbone,embeddings"   --dec_teacher_forcing
Namespace(seed=123, huggingface_id='byeonghwikim/abp_dataset', splits='/content/drive/MyDrive/mcr-agent/splits/1-shot.json', preprocess=False, pp_folder='pp', save_every_epoch=False, model='OEM.models.model.seq2seq_im_mask_obj', gpu=True, dout='/content/drive/MyDrive/mcr-agent/exp/OEM/ft_001', resume=None, finetune_from='/content/drive/MyDrive/mcr-agent/exp/OEM/latest.pth', freeze='encoder,visual_backbone,embeddings', reset_optim=False, use_templated_goals=False, use_streaming=True

In [ ]:
oem_log_dir = '/content/drive/MyDrive/mcr-agent/exp/OEM/ft_001'
oem_loss_data = analyze_tensorboard_logs(oem_log_dir)
print(oem_loss_data['seen'].tail())
print(oem_loss_data['unseen'].tail())

--- success load log from '/content/drive/MyDrive/mcr-agent/exp/OEM/ft_001' ---
      step              tag     value
6939  1652  valid/seen_loss  0.867692
6940  2065  valid/seen_loss  0.883635
6941  2478  valid/seen_loss  0.901933
6942  2891  valid/seen_loss  0.920428
6943  3304  valid/seen_loss  0.937311
      step                tag     value
7019  1652  valid/unseen_loss  0.152767
7020  2065  valid/unseen_loss  0.148822
7021  2478  valid/unseen_loss  0.144669
7022  2891  valid/unseen_loss  0.132620
7023  3304  valid/unseen_loss  0.138660


In [ ]:
!bash /content/mcr-agent/valid_seen.sh --object_model_path /content/drive/MyDrive/mcr-agent/exp/OEM/ft_001/latest.pth
!bash /content/mcr-agent/valid_unseen.sh --object_model_path /content/drive/MyDrive/mcr-agent/exp/OEM/ft_001/latest.pth

In [ ]:
# ==================== 5-shot fine-tune ====================
!bash /content/mcr-agent/OEM/finetune.sh \
  --ckpt /content/drive/MyDrive/mcr-agent/exp/OEM/latest.pth \
  --out  /content/drive/MyDrive/mcr-agent/exp/OEM/ft_005 \
  --model OEM.models.model.seq2seq_im_mask_obj \
  --splits /content/drive/MyDrive/mcr-agent/splits/5-shot.json \
  --batch 4 --epochs 10 --lr 1e-4 \
  --freeze "encoder,visual_backbone,embeddings" \
  --extra "--dec_teacher_forcing"

{'batch': 394, 'epoch': 0}
epoch:  10% 1/10 [08:04<1:12:42, 484.76s/it]{'batch': 394, 'epoch': 1}
epoch:  20% 2/10 [08:38<29:13, 219.22s/it]  {'batch': 394, 'epoch': 2}
epoch:  30% 3/10 [09:10<15:38, 134.04s/it]{'batch': 394, 'epoch': 3}
epoch:  40% 4/10 [09:43<09:24, 94.07s/it] {'batch': 394, 'epoch': 4}
epoch:  50% 5/10 [10:16<06:00, 72.12s/it]{'batch': 394, 'epoch': 5}
epoch:  60% 6/10 [10:49<03:55, 58.84s/it]{'batch': 394, 'epoch': 6}
epoch:  70% 7/10 [11:22<02:30, 50.27s/it]{'batch': 394, 'epoch': 7}
epoch:  80% 8/10 [11:55<01:29, 44.77s/it]{'batch': 394, 'epoch': 8}
epoch:  90% 9/10 [12:28<00:41, 41.01s/it]{'batch': 394, 'epoch': 9}
epoch: 100% 10/10 [13:01<00:00, 78.12s/it]


In [ ]:
oem_log_dir = '/content/drive/MyDrive/mcr-agent/exp/OEM/ft_005'
oem_loss_data = analyze_tensorboard_logs(oem_log_dir)
print(oem_loss_data['seen'].tail())
print(oem_loss_data['unseen'].tail())

--- success load log from '/content/drive/MyDrive/mcr-agent/exp/OEM/ft_005' ---
       step              tag     value
7885   9450  valid/seen_loss  0.391873
7886  11025  valid/seen_loss  0.424537
7887  12600  valid/seen_loss  0.434669
7888  14175  valid/seen_loss  0.432453
7889  15750  valid/seen_loss  0.459809
       step                tag     value
7895   9450  valid/unseen_loss  0.030202
7896  11025  valid/unseen_loss  0.018340
7897  12600  valid/unseen_loss  0.021554
7898  14175  valid/unseen_loss  0.025836
7899  15750  valid/unseen_loss  0.016330


In [ ]:
!bash /content/mcr-agent/valid_seen.sh --object_model_path /content/drive/MyDrive/mcr-agent/exp/OEM/ft_005/latest.pth
!bash /content/mcr-agent/valid_unseen.sh --object_model_path /content/drive/MyDrive/mcr-agent/exp/OEM/ft_005/latest.pth

In [152]:
# ==================== 10-shot fine-tune ====================
!bash /content/mcr-agent/OEM/finetune.sh \
  --ckpt /content/drive/MyDrive/mcr-agent/exp/OEM/latest.pth \
  --out  /content/drive/MyDrive/mcr-agent/exp/OEM/ft_010 \
  --model OEM.models.model.seq2seq_im_mask_obj \
  --splits /content/drive/MyDrive/mcr-agent/splits/10-shot.json \
  --batch 4 --epochs 10 --lr 1e-4 \
  --freeze "encoder,visual_backbone,embeddings" \
  --extra "--dec_teacher_forcing"

{'batch': 658, 'epoch': 1}
epoch:  20% 2/10 [13:46<46:43, 350.38s/it]  {'batch': 658, 'epoch': 2}
epoch:  30% 3/10 [14:42<25:13, 216.28s/it]{'batch': 658, 'epoch': 3}
epoch:  40% 4/10 [15:39<15:19, 153.27s/it]{'batch': 658, 'epoch': 4}
epoch:  50% 5/10 [16:36<09:52, 118.51s/it]{'batch': 658, 'epoch': 5}
epoch:  60% 6/10 [17:32<06:29, 97.37s/it] {'batch': 658, 'epoch': 6}
epoch:  70% 7/10 [18:30<04:12, 84.29s/it]{'batch': 658, 'epoch': 7}
epoch:  80% 8/10 [19:27<02:31, 75.56s/it]{'batch': 658, 'epoch': 8}
epoch:  90% 9/10 [20:24<01:09, 69.86s/it]{'batch': 658, 'epoch': 9}
epoch: 100% 10/10 [21:20<00:00, 128.07s/it]


In [153]:
oem_log_dir = '/content/drive/MyDrive/mcr-agent/exp/OEM/ft_010'
oem_loss_data = analyze_tensorboard_logs(oem_log_dir)
print(oem_loss_data['seen'].tail())
print(oem_loss_data['unseen'].tail())

--- success load log from '/content/drive/MyDrive/mcr-agent/exp/OEM/ft_010' ---
        step              tag     value
13165  15792  valid/seen_loss  0.275856
13166  18424  valid/seen_loss  0.258759
13167  21056  valid/seen_loss  0.260709
13168  23688  valid/seen_loss  0.264969
13169  26320  valid/seen_loss  0.273013
        step                tag     value
13175  15792  valid/unseen_loss  0.152737
13176  18424  valid/unseen_loss  0.288188
13177  21056  valid/unseen_loss  0.470037
13178  23688  valid/unseen_loss  0.433503
13179  26320  valid/unseen_loss  0.508002


In [ ]:
!bash /content/mcr-agent/valid_seen.sh --object_model_path /content/drive/MyDrive/mcr-agent/exp/OEM/ft_010/latest.pth
!bash /content/mcr-agent/valid_unseen.sh --object_model_path /content/drive/MyDrive/mcr-agent/exp/OEM/ft_010/latest.pth